# Before running the code please download holding file from zerodha. This code is compatible to zerodha holding file

In [ ]:
#Importing neccessary libraries
import dash
from dash import dash_table, dcc
import pandas as pd
import plotly.graph_objs as go
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

<div class="warning" style='padding:0.1em; background-color:#5CADE8; color:#090909'>
<span>
<p style='margin-top:1em; text-align:left'>
</p>
<p style='margin-left:1em;'>
</p><br>
    
**Sample Code for fetching stock details**
    In this code, we’re using the Ticker function from yfinance to get a variety of data for the specified stock (in this case, Reliance Industries on the National Stock Exchange of India). We then extract the info dictionary from this data which contains various fundamental details.
</span>
</div>

In [ ]:

import yfinance as yf

# Define the ticker symbol for the stock
tickerSymbol = 'MAXHEALTH.NS'  # Reliance Industries on NSE

# Get the data for this ticker
tickerData = yf.Ticker(tickerSymbol)

# Get the info dictionary containing various fundamental details
info_dict = tickerData.info

# Print the info dictionary
for key, value in info_dict.items():
    print(f"{key}: {value}")



<div class="warning" style='padding:0.1em; background-color:#5CADE8; color:#090909'>
<span>
<p style='margin-top:1em; text-align:left'>
</p>
<p style='margin-left:1em;'>
</p><br>
    
**Here we are taking input of holding.csv and generating plots for invested, current valuation**
</span>
</div>

In [ ]:
# Read data from CSV files
df_holdings = pd.read_csv('holdings.csv')

# Fetch sector and market cap information using yfinance
def fetch_info(symbol):
    tickerData = yf.Ticker(symbol + '.NS')  # Append '.NS' to the symbol
    info_dict = tickerData.info
    return info_dict.get('sector', None), info_dict.get('marketCap', None)

df_holdings['Sector'], df_holdings['Market Cap'] = zip(*df_holdings['Instrument'].apply(fetch_info))

# Fill None values with 0 and convert market cap to crores
df_holdings['Market Cap'].fillna(0, inplace=True)
df_holdings['Market Cap (in crores)'] = df_holdings['Market Cap'].astype(float) / 1e7

# Classify stocks based on market cap
def classify_stock(row):
    if row['Market Cap (in crores)'] <= 5000:
        return 'Small Cap'
    elif row['Market Cap (in crores)'] <= 20000:
        return 'Mid Cap'
    else:
        return 'Large Cap'

df_holdings['Cap'] = df_holdings.apply(classify_stock, axis=1)

# Sort dataframe by market cap
df_holdings.sort_values('Market Cap (in crores)', ascending=False, inplace=True)

# Create labels for the plot
df_holdings['Label'] = df_holdings['Instrument'] + ' - ' + df_holdings['Sector'] + ' - ' + df_holdings['Cap']

# Calculate invested amount
df_holdings['Invested Amount'] = df_holdings['Avg. cost'] * df_holdings['Qty.']

# Create donut plot for portfolio invested analysis 
fig1 = go.Figure(data=[go.Pie(labels=df_holdings['Label'], values=df_holdings['Invested Amount'], hole=0.3)])
fig1.update_layout(
    title_text="Portfolio Analysis - Invested Amount",
    height=700,  # Adjust as needed
    width=1000    # Adjust as needed
)


# Create donut plot for portfolio current analysis 
fig2 = go.Figure(data=[go.Pie(labels=df_holdings['Label'], values=df_holdings['Cur. val'], hole=0.3)])
fig2.update_layout(
    title_text="Portfolio Analysis - Current Value",
    height=600,  # Adjust as needed
    width=1000    # Adjust as needed
)

# Create donut plot for sector analysis
sector_data = df_holdings.groupby('Sector')['Invested Amount'].sum().reset_index()
fig3 = go.Figure(data=[go.Pie(labels=sector_data['Sector'], values=sector_data['Invested Amount'], hole=0.3)])
fig3.update_layout(
    title_text="Portfolio Analysis - Sectors",
    height=600,  # Adjust as needed
    width=600    # Adjust as needed
)

# Create donut plot for market cap analysis
market_cap_data = df_holdings.groupby('Cap')['Invested Amount'].sum().reset_index()
fig4 = go.Figure(data=[go.Pie(labels=market_cap_data['Cap'], values=market_cap_data['Invested Amount'], hole=0.3)])
fig4.update_layout(
    title_text="Portfolio Analysis - Total Holding in each Market Capitalization",
    height=600,  # Adjust as needed
    width=600    # Adjust as needed
)

# Add the new plot to the Dash app layout
app = dash.Dash(__name__)

app.layout = dash.html.Div([
    dash_table.DataTable(
        id='table',
        columns=[{"name": i, "id": i} for i in df_holdings.columns],
        data=df_holdings.to_dict('records'),
    ),
    dcc.Graph(figure=fig1),
    dcc.Graph(figure=fig2)
])

if __name__ == '__main__':
    app.run_server(debug=True)
    


<div class="warning" style='padding:0.1em; background-color:#5CADE8; color:#090909'>
<span>
<p style='margin-top:1em; text-align:left'>
</p>
<p style='margin-left:1em;'>
</p><br>
    
**Here we are generating plots sectorwise, size of company**
</span>
</div>

In [ ]:
# Add the new plot to the Dash app layout
app = dash.Dash(__name__)

app.layout = dash.html.Div([
    dash_table.DataTable(
        id='table',
        columns=[{"name": i, "id": i} for i in df_holdings.columns],
        data=df_holdings.to_dict('records'),
    ),
    dcc.Graph(figure=fig3),  # New plot
    dcc.Graph(figure=fig4)  # New plot
])

if __name__ == '__main__':
    app.run_server(debug=True)

Below code will help you to rebalance your portfolio. Few changes need to done such as comment ETF holding 'CPSEETF', 'SETF10GILT', 'GOLDBEES', 'MID150BEES', and 'NIFTYBEES' if not in your portfolio. Read comment carefully

In [ ]:
# Fetch sector information using yfinance
def fetch_info(symbol):
    tickerData = yf.Ticker(symbol + '.NS')  # Append '.NS' to the symbol
    info_dict = tickerData.info
    if 'regularMarketPrice' not in info_dict:
        # Fetch the current price using the 'history' method if 'regularMarketPrice' is not available
        info_dict['regularMarketPrice'] = tickerData.history(period='1d').tail(1)['Close'].iloc[0]
    return info_dict

# Apply the function to each instrument in the holdings
df_holdings['Info'] = df_holdings['Instrument'].apply(fetch_info)

# Now, you can extract any information you need from the 'Info' column. For example:
df_holdings['Sector'] = df_holdings['Info'].apply(lambda info: info.get('sector', 'Unknown'))
df_holdings['PE_Ratio'] = df_holdings['Info'].apply(lambda info: info.get('trailingPE', 0))
df_holdings['Beta'] = df_holdings['Info'].apply(lambda info: info.get('beta', 0))
df_holdings['DebtToEquity'] = df_holdings['Info'].apply(lambda info: info.get('debtToEquity', 0))
df_holdings['ReturnOnEquity'] = df_holdings['Info'].apply(lambda info: info.get('returnOnEquity', 0))
df_holdings['EarningsGrowth'] = df_holdings['Info'].apply(lambda info: info.get('earningsGrowth', 0))
df_holdings['RevenueGrowth'] = df_holdings['Info'].apply(lambda info: info.get('revenueGrowth', 0))
df_holdings['MarketCap'] = df_holdings['Info'].apply(lambda info: info.get('marketCap', 0))
df_holdings['CurrentPrice'] = df_holdings['Info'].apply(lambda info: info.get('regularMarketPrice', 0))
df_holdings['ForwardPE'] = df_holdings['Info'].apply(lambda info: info.get('forwardPE', 0))
df_holdings['PriceToBook'] = df_holdings['Info'].apply(lambda info: info.get('priceToBook', 0))
df_holdings['TrailingEPS'] = df_holdings['Info'].apply(lambda info: info.get('trailingEps', 0))
df_holdings['ForwardEPS'] = df_holdings['Info'].apply(lambda info: info.get('forwardEps', 0))
df_holdings['EarningsGrowth'] = df_holdings['Info'].apply(lambda info: info.get('earningsGrowth', 0))
df_holdings['RevenueGrowth'] = df_holdings['Info'].apply(lambda info: info.get('revenueGrowth', 0))
df_holdings['RecommendationMean'] = df_holdings['Info'].apply(lambda info: info.get('recommendationMean', 0))
df_holdings['RecommendationKey'] = df_holdings['Info'].apply(lambda info: info.get('recommendationKey', 0))
df_holdings['HeldPercentInsiders'] = df_holdings['Info'].apply(lambda info: info.get('heldPercentInsiders', 0))
df_holdings['HeldPercentInstitutions'] = df_holdings['Info'].apply(lambda info: info.get('heldPercentInstitutions', 0))


# Add the new parameters to the list of metrics to normalize
metrics_to_normalize = ['PE_Ratio', 'Beta', 'DebtToEquity', 'ReturnOnEquity', 'EarningsGrowth', 'RevenueGrowth', 'MarketCap', 'ForwardPE', 'PriceToBook', 'ForwardEPS']

# Use RobustScaler for normalization
scaler = RobustScaler()
df_holdings[metrics_to_normalize] = scaler.fit_transform(df_holdings[metrics_to_normalize])

# Calculate weights (here, we simply sum the normalized metrics, but you could use a different strategy)
df_holdings['Weight'] = df_holdings[metrics_to_normalize].sum(axis=1)
df_holdings['Weight'] = df_holdings['Weight'] / df_holdings['Weight'].sum()  # Ensure weights sum to 1

# Set specific target allocations for 'CPSEETF', 'SETF10GILT', 'GOLDBEES', 'MID150BEES', and 'NIFTYBEES'

df_holdings.loc[df_holdings['Instrument'] == 'CPSEETF', 'Weight'] = 0.02 #Comment if not in your portfolio
df_holdings.loc[df_holdings['Instrument'] == 'ITBEES', 'Weight'] = 0.03 #Comment if not in your portfolio
df_holdings.loc[df_holdings['Instrument'] == 'SETF10GILT', 'Weight'] = 0.10 #Comment if not in your portfolio
df_holdings.loc[df_holdings['Instrument'] == 'GOLDBEES', 'Weight'] = 0.10 #Comment if not in your portfolio
df_holdings.loc[df_holdings['Instrument'] == 'MID150BEES', 'Weight'] = 0.15 #Comment if not in your portfolio
df_holdings.loc[df_holdings['Instrument'] == 'NIFTYBEES', 'Weight'] = 0.25 #Comment if not in your portfolio


# Adjust the weights of the other stocks so that the total sums to 1
other_stocks = df_holdings['Instrument'].isin(['CPSEETF', 'ITBEES', 'SETF10GILT', 'GOLDBEES', 'MID150BEES', 'NIFTYBEES']) #Comment if not in your portfolio
df_holdings.loc[~other_stocks, 'Weight'] = df_holdings.loc[~other_stocks, 'Weight'] / df_holdings.loc[~other_stocks, 'Weight'].sum() * (1 - df_holdings.loc[other_stocks, 'Weight'].sum())

# Give more weight to certain sectors
sector_weights = {'Financial Services': 1.5, 'Technology': 1.0, 'Healthcare': 1.2}  # Adjust these values as needed
df_holdings['Weight'] = df_holdings.apply(lambda row: row['Weight'] * sector_weights.get(row['Sector'], 1), axis=1)
df_holdings['Weight'] = df_holdings['Weight'] / df_holdings['Weight'].sum()  # Ensure weights sum to 1

def rebalance_portfolio(df):
    # Calculate current value of portfolio
    total_value = df['Cur. val'].sum()

    # Calculate current allocations
    df['Current Allocation'] = df['Cur. val'] / total_value

    # Calculate target value for each instrument
    df['Target Value'] = total_value * df['Weight']

    # Calculate the difference between current value and target value
    df['Difference'] = df['Target Value'] - df['Cur. val']

    # Calculate the number of shares to buy or sell
    df['Shares to Buy/Sell'] = df['Difference'] / df['CurrentPrice']

    return df

# Rebalance portfolio
df_holdings = rebalance_portfolio(df_holdings)

print(df_holdings[['Instrument', 'CurrentPrice', 'Weight', 'Current Allocation', 'Target Value', 'Difference', 'Shares to Buy/Sell', 'RecommendationKey', 'HeldPercentInsiders', 'HeldPercentInstitutions']])


The ‘Difference’ column in your output indicates how much value you need to add or remove from each stock to achieve the target allocation. Here’s how you can interpret it:

If the ‘Difference’ is positive, it means you need to buy more of that stock. The value indicates how much more you need to buy.
If the ‘Difference’ is negative, it means you need to sell some of that stock. The absolute value indicates how much you need to sell.
For example, looking at the first row for ‘AFFLE’, the ‘Difference’ is 2315.401170. This means you need to buy additional ‘AFFLE’ stocks worth that amount to achieve your target allocation.

On the other hand, for ‘ALKYLAMINE’, the ‘Difference’ is -1105.724734. This means you need to sell ‘ALKYLAMINE’ stocks worth the absolute value of that amount.

Please note that these calculations are based on the current prices of the stocks. The actual amount you need to buy or sell could vary depending on the price at the time of transaction. Also, transaction costs are not considered in these calculations.

In [ ]:
import datetime

# Get today's date
today = datetime.date.today()

# Convert the date to a string in the format YYYYMMDD
date_str = today.strftime('%Y%m%d')

# Use the date string in the filename
df_holdings[['Instrument', 'CurrentPrice', 'Weight', 'Current Allocation', 'Target Value', 'Difference', 'Shares to Buy/Sell', 'RecommendationMean', 'RecommendationKey', 'HeldPercentInsiders', 'HeldPercentInstitutions']].to_csv(f'rebalanced_portfolio_{date_str}.csv', index=False)

The number of shares you need to buy (if the value is positive) or sell (if the value is negative) for each instrument in your portfolio. Please note that this calculation assumes that you can buy or sell fractional shares. If your trading platform only allows whole share transactions, you may need to round the values to the nearest whole number

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
# Assuming df_holdings is your DataFrame and it's already defined
df_encoded = pd.get_dummies(df_holdings, columns=['Sector'])

# Selecting the required columns
selected_columns = ['PE_Ratio', 'Beta', 'DebtToEquity', 'ReturnOnEquity', 'EarningsGrowth', 'RevenueGrowth', 'MarketCap', 'ForwardPE', 'PriceToBook', 'ForwardEPS']
#selected_columns += [col for col in df_encoded.columns if 'Sector_' in col]  # Adding the encoded 'Sector' columns

df_selected = df_encoded[selected_columns]

# Calculating the correlation matrix
corr_matrix = df_selected.corr()

# Creating the correlation heatmap
plt.figure(figsize=(14, 10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
print(df_holdings[['Instrument', 'Avg. cost', 'Qty.', 'Invested Amount']])


**Next we can check the performance of our portfolio with in the index Nifty50**

In [ ]:

# Initialize a DataFrame to store the historical data
df_hist = pd.DataFrame()
# Fetch the historical data
for ticker in df_holdings['Instrument']:
    tickerData = yf.Ticker(ticker + '.NS')
    tickerDf = tickerData.history(period='1d', start='2023-02-01', end='2024-02-01') #Change the date range accordingly
    if tickerDf.empty:
        print(f"No data available for {ticker} for the specified date range")
        continue
    df_hist[ticker] = tickerDf['Close']

# Replace NaN values with 0
#df_hist.fillna(0, inplace=True)


# Check if the DataFrame is not empty
if not df_hist.empty:
    # Calculate the sum of the instrument prices for each day
    df_hist['Sum'] = df_hist.sum(axis=1)

    # Fetch the historical data for Nifty 50
    nifty50 = yf.Ticker('^NSEI').history(period='1d', start='2023-02-01', end='2024-02-01')['Close'] #Change the date range accordingly

    # Normalize the data
    df_hist = df_hist / df_hist.iloc[0]
    nifty50 = nifty50 / nifty50.iloc[0]

    # Plot the data
    plt.figure(figsize=(14, 7))
    plt.plot(df_hist.index, df_hist['Sum'], label='Sum of Instruments')
    plt.plot(nifty50.index, nifty50, label='Nifty 50')
    plt.legend()
    plt.show()
else:
    print("No data available for the specified date range for any of the instruments")